In [1]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai
!pip install "tracing-auto-instrumentation[llama-index]" --upgrade

In [2]:
import llama_index.core

from tracing_auto_instrumentation.llama_index import LlamaIndexCallbackHandler
llama_index.core.global_handler = LlamaIndexCallbackHandler(
    project_name="Function call demo",
)

In [3]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.tools import FunctionTool

import os
import dotenv
from getpass import getpass

dotenv.load_dotenv()
if os.getenv("OPENAI_API_KEY") is None:
    os.environ["OPENAI_API_KEY"] = getpass(
        "Paste your OpenAI key from:"
        " https://platform.openai.com/account/api-keys\n"
    )
assert os.getenv("OPENAI_API_KEY", "").startswith(
    "sk-"
), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


In [4]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

llm = OpenAI(model="gpt-3.5-turbo-1106")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [5]:
# Note: if you have a APIConnectionError and are on a Mac Silicon device,
# you may need to run the following command in your terminal:
# bash /Applications/Python*/Install\ Certificates.command

# After running the command above, please restart this notebook and try again

response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

[DEBUG] 2024-06-12 16:30:06,610 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-06-12 16:30:06,610 _config.py:146: load_verify_locations cafile='/opt/homebrew/Caskroom/miniconda/base/envs/eval/lib/python3.12/site-packages/certifi/cacert.pem'
[DEBUG] 2024-06-12 16:30:06,618 _base_client.py:446: Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'role': 'user', 'content': 'What is (121 * 3) + 42?'}], 'model': 'gpt-3.5-turbo-1106', 'stream': False, 'temperature': 0.1, 'tool_choice': 'auto', 'tools': [{'type': 'function', 'function': {'name': 'multiply', 'description': 'multiply(a: int, b: int) -> int\nMultiple two integers and returns the result integer', 'parameters': {'type': 'object', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b']}}}, {'type': 'function', 'function': {'name': 'add', 'description': 'add(a: int, b: i

Added user message to memory: What is (121 * 3) + 42?


[DEBUG] 2024-06-12 16:30:07,923 _trace.py:45: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 12 Jun 2024 20:30:07 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'844'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999977'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_1abeb954219dc0c44b8bdae5212c57c2'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Set-Cookie', b'__cf_bm=bqxfstpty_lTtTYsX2bjwsi1HAHdWO1VSCi3yjB8vk0-1718224207-1.0.1.1-1yXSkXQMqcnEHzlz8TDdzEsiLrAjqXOYV2fIzVl3gQHMdCBOTwBn9U5cy8BWCV8PEB82OqMxrfhyOLjn9UQuiA; path=/; expir

=== Calling Function ===
Calling function: multiply with args: {"a": 121, "b": 3}
Got output: 363

=== Calling Function ===
Calling function: add with args: {"a": 363, "b": 42}
Got output: 405



[DEBUG] 2024-06-12 16:30:09,255 _trace.py:45: receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Date', b'Wed, 12 Jun 2024 20:30:09 GMT'), (b'Content-Type', b'application/json'), (b'Transfer-Encoding', b'chunked'), (b'Connection', b'keep-alive'), (b'openai-organization', b'lastmile-ai'), (b'openai-processing-ms', b'696'), (b'openai-version', b'2020-10-01'), (b'strict-transport-security', b'max-age=15724800; includeSubDomains'), (b'x-ratelimit-limit-requests', b'10000'), (b'x-ratelimit-limit-tokens', b'2000000'), (b'x-ratelimit-remaining-requests', b'9999'), (b'x-ratelimit-remaining-tokens', b'1999971'), (b'x-ratelimit-reset-requests', b'6ms'), (b'x-ratelimit-reset-tokens', b'0s'), (b'x-request-id', b'req_84ddc098548fcfb22ffc50e3e19883c5'), (b'CF-Cache-Status', b'DYNAMIC'), (b'Server', b'cloudflare'), (b'CF-RAY', b'892c98d4fa797cb2-EWR'), (b'Content-Encoding', b'gzip'), (b'alt-svc', b'h3=":443"; ma=86400')])
[INFO] 2024-06-12 16:30:09,256 _client.py:1026: HTTP 

The result of (121 * 3) is 363, and when you add 42 to it, you get 405.
